# Naive Monte Carlo in a Black-Scholes Economy


$$
S_{T} = S_{0} \exp{\left((r - \delta  + \frac{1}{2} \sigma^{2}) T + \sigma \sqrt{T} Z \right)}
$$

In [1]:
import numpy as np

In [2]:
z = np.random.normal(size = 100)

In [3]:
z

array([-1.01104245,  1.43889453,  0.0528921 , -1.11410316, -0.48872575,
       -0.45149172, -0.6464181 , -0.22310843, -0.56736406, -0.14128486,
        0.39150964,  0.12263835,  0.07013858,  1.2347408 ,  0.29387234,
       -0.24191727,  0.75931928,  0.30055797, -0.23709423, -1.22224657,
       -0.76723565,  0.46980358,  0.12594491,  3.18639146,  1.5997342 ,
        1.23486254, -1.25915547,  0.49700924,  0.74503026, -1.05961457,
        0.06397939, -0.23163888,  0.27616737, -0.35333015,  0.62236459,
        0.17417758,  1.56902721,  0.76872647,  0.09901278, -2.36433827,
       -1.22054525,  1.53167879, -0.36847256,  1.66342486, -0.03453464,
       -0.29459828, -0.79820531, -0.85955693, -0.77723422,  1.03572223,
       -1.37784644, -2.00228055, -0.86548304, -1.01379805,  1.30246518,
       -1.2402949 ,  0.47619783, -0.52426797, -0.72680162,  0.72098335,
        0.34695564,  0.07916925, -0.30332722,  0.02449731, -0.52158831,
        1.25821063, -0.01129192, -2.10093905,  0.58148867,  1.53

## Set Up Parameters

In [4]:
S = 41.0
K = 40.0
r = 0.08
v = 0.30
T = 1.0
q = 0.0


In [5]:
M = 1000000
spot_t = np.empty((M,))


In [6]:
nudt = (r - q - 0.5 * v * v) * T
sigdt = v * np.sqrt(T)
z = np.random.normal(size=(M,))

In [7]:
#for i in range(M):
#    spot_t[i] = S * np.exp(nudt + sigdt * z[i])

In [8]:
spot_t = S * np.exp(nudt + sigdt * z)

In [9]:
spot_t[:10]

array([ 76.75410857,  35.17374881,  46.70116031,  58.53424105,
        44.45859743,  51.323376  ,  43.28661671,  41.17173452,
        51.16938619,  48.93720828])

In [10]:
def CallPayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)


def PutPayoff(spot, strike):
    return np.maximum(strike - spot, 0.0)

In [11]:
call_t = CallPayoff(spot_t, K)

In [12]:
call_t[:10]

array([ 36.75410857,   0.        ,   6.70116031,  18.53424105,
         4.45859743,  11.323376  ,   3.28661671,   1.17173452,
        11.16938619,   8.93720828])

In [13]:
bob = call_t.mean()

In [14]:
bob

7.5435420759515122

In [15]:
callPrc = np.exp(-r * T) * bob

In [16]:
callPrc

6.9635669999662175

In [17]:
put_t = PutPayoff(spot_t, K)

In [18]:
put_t[:10]

array([ 0.        ,  4.82625119,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [19]:
putPrc = np.exp(-r * T) * put_t.mean()

In [20]:
putPrc

2.8929095170635484

## Simulating Paths

In [36]:
reps = 3000000
steps = 4
path = np.zeros((reps,steps))

In [37]:
path
path[:,0] = S
path

array([[ 41.,   0.,   0.,   0.],
       [ 41.,   0.,   0.,   0.],
       [ 41.,   0.,   0.,   0.],
       ..., 
       [ 41.,   0.,   0.,   0.],
       [ 41.,   0.,   0.,   0.],
       [ 41.,   0.,   0.,   0.]])

In [38]:
dt = T / steps
nudt = (r - q - 0.5 * v * v) * dt
sigdt = v * np.sqrt(dt)

In [39]:

for i in range(reps):
    z = np.random.normal(size=steps)
    for j in range(1, steps):
        path[i,j] = path[i,j-1] * np.exp(nudt + sigdt * z[j])
        
path

array([[ 41.        ,  35.37683475,  28.6394618 ,  22.97888409],
       [ 41.        ,  49.95862171,  58.87976882,  67.93253433],
       [ 41.        ,  35.68488534,  37.51075424,  37.48990806],
       ..., 
       [ 41.        ,  33.78934329,  51.24500982,  48.53897259],
       [ 41.        ,  37.90424041,  51.55999163,  52.15236133],
       [ 41.        ,  43.25988887,  49.20681518,  55.49915372]])

In [40]:
def ArithmeticAsianCallPayoff(path, strike):
    spot_t = path.mean(axis = 1)
    return np.maximum(spot_t - strike, 0.0)

In [41]:
call_t = ArithmeticAsianCallPayoff(path, K).mean()
call_t *= np.exp(-r * T)
call_t

3.3325571553321245